# Visualization of NCOPID-19 Data
One of the reasons of Python, such populay by ML developers, is its visualization utilities:
- Matplotlib, provides the basic functions and utilities to make visualization;
- seaborn provides high-level interface for drawing  informative statistical graphics;
- plotly, it could not be absent of creating both on-line and off-line visualizations with hand-on interact.

Furthermore, python never let you down if animation, dashboard setup are required; try `moviepy, ipywidget, dash, etc`.

**Note**. Installing or updating the Python package, you could do it as follows:

```
 shell > pip install -U plotly
   or
 shell > conda install plotly
```
**Exercise**: install plotly as above.


## Data Prepatation
In last week practicing, we had learn how to work on the time series data from [JUH](https://github.com/CSSEGISandData/COVID-19). Now create today practicing, NCov-2.ipynb, as follows:

```
        COVID-19-master/
           csse_covid_19_data/
           ...        
        t/
           NCov-1.ipynb
           NCov-2.ipynb
           ...
           tmp/
``` 
In this practicing, the daily data would be used, but not time-series data.        

In [ ]:
import pandas as pd
import numpy as np
import re
import os

import pickle
import os.path
from datetime import datetime, date, time 
from time import strftime
from tqdm import tqdm

import json

# use to parse the data timestamp
from dateutil.parser import parse

import matplotlib.pyplot as plt

In [ ]:
# What day is today
str(datetime.date(datetime.now()))

In [ ]:
df=pd.read_csv('../COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/03-20-2020.csv')
df.head(6)

Let us to look at the last daily data:
```
Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude
Hubei,Mainland China,<span class="burk">2020-03-08</span>T<span class="girk">14:43:03</span>,67707,2986,45235,30.9756,112.2707
,Italy,2020-03-08T18:03:04,7375,366,622,43.0000,12.0000
,South Korea,2020-03-08T12:53:03,7314,50,118,36.0000,128.0000
,Iran,2020-03-08T11:03:30,6566,194,2134,32.0000,53.0000
Guangdong,Mainland China,2020-03-08T14:43:03,1352,7,1256,23.3417,113.4244  
...
```
1. **the first line**: the name of each column;
- **from second line**, each data was recorded as one line and each column was seperted by comma symbol '`,`';
-  data in `Last Update`, the 4th column, were in standard datetime format, `Year-month-date time`, where `T` means white space; lately, use the following to parse the data and get `date` and `time`: 

<table style="width: 100%">
      <tbody><tr>
        <td style="text-align: center; padding-left: 0em; padding-right: 0em"><table style="display: inline-table; vertical-align: middle">
          <tbody><tr>
            <td><span style="margin-left: 2em"></span>[0]</td>
            <td></td>
            <td><span style="margin-left: 1em"></span>[1]</td>
          </tr><tr>
            <td><font color="red">2020-03-08</font></td>
            <td>T</td>
            <td><font color="blue">14:43:03</font></td>
          </tr><tr>
            <td><span style="margin-left: 2em"></span><font color="red">date</font></td>
            <td></td>
            <td><span style="margin-left: 1em"></span><font color="blue">time</font></td>
          </tr></tbody>
        </table></td>
      </tr><tr>
        <td style="text-align: center; padding-left: 0em; padding-right: 0em; height: 0.5em"></td>
      </tr></tbody>
    </table>



   ```
   date=parse(str(last_update).split(' ')[0]).strftime("%Y-%m-%d")
   time=parse(str(last_update).split(' ')[1]).strftime("%Y-%m-%d")
   ```
- the first column in third row is in blank which means being omitted generally; it is `NaN` in Pandas.

Now load the csv data into worksheet by pandas as before:

In [ ]:
DATA = '../COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'

def clean_sheet_names(new_ranges):
    '''
    Get rid of the duplicate sheets, only take the sheets from the 
    latest point in the day
    '''
    indices = []
    
    # Remove all sheets that dont have a numeric header
    numeric_sheets = [x for x in new_ranges if re.search(r'\d', x)]  
    
    return numeric_sheets

In [ ]:
os.listdir(DATA)

csv files in 

     ../COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/

and beginning with a numeric character are data but `README.md` and `.gitignore` are not.

In [ ]:
sheets = os.listdir(DATA)

# Clean the result to the sheet tabs we want
cleaned_sheets = clean_sheet_names(sorted(sheets, reverse=True))

In [ ]:
cleaned_sheets

In [ ]:
'''
For assigning date by the time sheet name
'''

def clean_last_updates(last_update):
    date = parse(str(last_update).split(' ')[0]).strftime("%Y-%m-%d")
    time = parse(str(last_update).split(' ')[1]).strftime('%H:%M:%S')
    parsed_date = str(date) + ' ' + str(time)

    return parsed_date

def get_date(last_update):
    return parse(str(last_update).split(' ')[0]).strftime("%Y-%m-%d")
def get_csv_date(file):
    return get_date(file.split('.')[0] + ' ') 

def drop_duplicates(df_):
    '''
    Take the max date value for each province for a given date
    '''
    days_list = []
    
    for datetime in df_.Date.unique():
        tmp_df = df_[df_.Date == datetime]
        tmp_df = tmp_df.sort_values(['Last Update']).drop_duplicates('Province/State', keep='last')
        days_list.append(tmp_df)

    return days_list

In [ ]:
f=cleaned_sheets

In [ ]:
f[0]>'02-21-2020.csv'

In [ ]:
tmp_df = pd.read_csv(os.path.join(DATA, '03-22-2020.csv'), index_col=None, header=0, parse_dates=['Last_Update'])
tmp_df.rename(columns={"Last_Update": "Last Update"},inplace=True) 
tmp_df.head()

In [ ]:
keep_cols = ['Confirmed', 'Country/Region', 'Deaths', 'Last Update', 'Province/State', 'Recovered']
numeric_cols = ['Confirmed', 'Deaths', 'Recovered']

def get_data(cleaned_sheets):
    all_csv = []
    # Import all CSV's
    #for file in sorted(sheets):
    for file in tqdm(sorted(sheets), desc='... importing data: '):    
        if 'csv' in file:
            #print( file)
            if (file>'03-21-2020.csv'):
                Last_Date='Last_Update'
            else:
                Last_Date='Last Update'
            tmp_df = pd.read_csv(os.path.join(DATA, file), index_col=None, header=0, parse_dates=[Last_Date])
            if (file>'03-21-2020.csv'):
               tmp_df.rename(columns={"Last_Update": "Last Update",'Country_Region':'Country/Region', 'Province_State': 'Province/State'},inplace=True) 
            
            tmp_df = tmp_df[keep_cols]
            tmp_df[numeric_cols] = tmp_df[numeric_cols].fillna(0)
            tmp_df[numeric_cols] = tmp_df[numeric_cols].astype(int)
            tmp_df['Province/State'].fillna(tmp_df['Country/Region'], inplace=True)
            #  tmp_df['Last_Update'] = tmp_df['Last_Update'].apply(clean_last_updates)
            #else:
            #   tmp_df['Last Update'] = tmp_df['Last Update'].apply(clean_last_updates)
            tmp_df['Date'] = tmp_df['Last Update'].apply(get_date)
            tmp_df['file_date'] = get_csv_date(file)
            all_csv.append(tmp_df)

    df_ = pd.concat(all_csv, axis=0, ignore_index=True, sort=True)
    df_ = df_.sort_values(by=['Last Update'])

    #Get the last entry per region by date
    frames = drop_duplicates(df_)
    tmp = pd.concat(frames, axis=0, ignore_index=True, sort=True)
    print("\nData contenance complete, total %d daily data used..." % len(sheets))
    
    return tmp

df = get_data(cleaned_sheets)

In [ ]:
df.tail()

In [ ]:
# Now that we have all the data we now need to clean it 
# - Fill null values
# - remore suspected values
# - change column names
def clean_data(tmp_df):
    if 'Demised' in tmp_df.columns:
        tmp_df.rename(columns={'Demised':'Deaths'}, inplace=True)

    if 'Country/Region' in tmp_df.columns:
        tmp_df.rename(columns={'Country/Region':'country'}, inplace=True)
    
    if 'Province/State' in tmp_df.columns:
        tmp_df.rename(columns={'Province/State':'province'}, inplace=True)
        
    if 'Last Update' in tmp_df.columns:
        tmp_df.rename(columns={'Last Update':'datetime'}, inplace=True)
        
    if 'Suspected' in tmp_df.columns:
        tmp_df = tmp_df.drop(columns='Suspected')

    for col in tmp_df.columns:
        tmp_df[col] = tmp_df[col].fillna(0)
    
    #Lower case all col names
    tmp_df.columns = map(str.lower, tmp_df.columns) 
    return tmp_df

df  = clean_data(df)

In [ ]:
df.country

In [ ]:
country_list = list(map(lambda x:x.lower().strip(), set(df.country.values)))


In [ ]:
from difflib import get_close_matches

COUNTRY = 'taiwan*'

def get_similar_countries(c, country_list):
    pos_countries = get_close_matches(c, country_list)
    
    if len(pos_countries) > 0:
        print(c, 'was not listed. did you mean', pos_countries[0].capitalize() + '?')
        sys.exit()
    else:
        print(c, 'was not listed.')
        sys.exit()
        
def check_specified_country(df):
    if COUNTRY:
        print('Country specified')
        if COUNTRY.lower() == 'china':
            print(COUNTRY, 'was not listed. did you mean Mainland China?')
            
        elif COUNTRY.lower() not in country_list:
            get_similar_countries(COUNTRY, country_list)
            
        else:
            print('... filtering data for', COUNTRY)
            if len(COUNTRY) == 2:
                df = df[df.country == COUNTRY.upper()]
            else:
                df = df[df.country == COUNTRY.capitalize()]
            return df
    else:
        print('No specific country specified')
        return df

In [ ]:
df = check_specified_country(df)

In [ ]:
# sheets need to be sorted by date value
print('Sorting by datetime...')
current_date = str(datetime.date(datetime.now()))

if df.date.max() == current_date:
    df = df[df.date != df.date.max()]
else:
    df = df[df.date != current_date]

df = df.sort_values('datetime')

In [ ]:
df

In [ ]:
'''
Get the difference of the sum totals for each
date and plot them on a trendline graph
'''
def get_new_cases(tmp, col):
    diff_list = []
    tmp_df_list = []
    df = tmp.copy()

    for i, day in enumerate(df.sort_values('date').date.unique()):    
        tmp_df = df[df.date == day]
        tmp_df_list.append(tmp_df[col].sum())
        
        if i == 0:
            diff_list.append(tmp_df[col].sum())
        else:
            diff_list.append(tmp_df[col].sum() - tmp_df_list[i-1])
        
    return diff_list

def get_moving_average(tmp, col):
    df = tmp.copy()
    return df[col].rolling(window=2).mean()

def get_exp_moving_average(tmp, col):
    df = tmp.copy()
    return df[col].ewm(span=2, adjust=True).mean()

In [ ]:
print('Calculating dataframe for new cases...')
daily_cases_df = pd.DataFrame([])
daily_cases_df['new_confirmed_cases'] = get_new_cases(df, 'confirmed')
daily_cases_df['new_deaths'] = get_new_cases(df, 'deaths')
daily_cases_df['new_recoveries'] = get_new_cases(df, 'recovered')
daily_cases_df['date'] = df.date.unique()

#Moving average
daily_cases_df['confirmed_MA'] = get_moving_average(daily_cases_df, 'new_confirmed_cases')
daily_cases_df['deaths_MA'] = get_moving_average(daily_cases_df, 'new_deaths')
daily_cases_df['recovered_MA'] = get_moving_average(daily_cases_df, 'new_recoveries')

#Exponential moving average
daily_cases_df['confirmed_exp_MA'] = get_exp_moving_average(daily_cases_df, 'new_confirmed_cases')
daily_cases_df['deaths_exp_MA'] = get_exp_moving_average(daily_cases_df, 'new_deaths')
daily_cases_df['recovered_exp_MA'] = get_exp_moving_average(daily_cases_df, 'new_recoveries')

In [ ]:
'''
Calculate the number of people that are ACTUALLY infected on a given day
currently infected = sum of people date - (recovored + died)
ex: 5 = 10 - (4 - 1)

'''
current_infected = pd.DataFrame([])
current_infected['currently_infected'] = (df.groupby('date').confirmed.sum() - \
                                          (df.groupby('date').deaths.sum() + df.groupby('date').recovered.sum()))
current_infected['delta'] = (current_infected['currently_infected'] - df.groupby('date').confirmed.sum())
daily_cases_df = pd.merge(daily_cases_df, current_infected, how='outer', on='date')

Create a new folder, named `data`, here manually:

In [ ]:
#Create date of extraction folder
#data_folder = os.path.join('data', str(datetime.date(datetime.now())))
#save_dir = os.path.join(out, data_folder)

#if not os.path.exists(save_dir):
#    os.system('mkdir -p ' + save_dir)
save_dir='data'
#print('Creating subdirectory for data...')
#print('...', save_dir)

print('Saving...')
file_name = 'agg_data_{}.parquet.gzip'.format(datetime.date(datetime.now()))
df.astype(str).to_parquet(os.path.join(save_dir, file_name), compression='gzip')
print('...', file_name)


csv_file_name = 'agg_data_{}.csv'.format(datetime.date(datetime.now()))
df.astype(str).to_csv(os.path.join(save_dir, csv_file_name))
print('...', csv_file_name)


daily_cases_file_name = 'trend_{}.csv'.format(datetime.date(datetime.now()))
daily_cases_df.astype(str).to_csv(os.path.join(save_dir, daily_cases_file_name))
print('...', daily_cases_file_name)

print('Done!')

In [ ]:
from plotly.offline import init_notebook_mode, plot, iplot, download_plotlyjs
import plotly.graph_objs as go
import plotly.express as px

import plotly.io as pio

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.log1p(df['confirmed']), y=df['file_date'],
                    mode='lines+markers',
                    name='Confirmed in Log'))
fig.add_trace(go.Scatter(x=np.log1p(df['recovered']), y=df['file_date'],
                    mode='lines+markers',
                    name='Recovered'))

plot(fig, filename='NCOVID-19.html')